In [ ]:
# Install Python libraries
!pip install google-generativeai pandas matplotlib

In [ ]:
# Import libraries
from google.colab import userdata  # Access user data (for API key storage)
import os  # OS-related operations
import pandas as pd  # For handling tabular data
import google.generativeai as genai  # Google Gemini API for AI-powered log analysis
import matplotlib.pyplot as plt  # For data visualization
from google.colab import files  # To handle file uploads/downloads

# Retrieve the API key from Google Colab's secure storage
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the Generative AI API with the provided key
genai.configure(api_key=GOOGLE_API_KEY)

# Upload firewall log file from local system
uploaded = files.upload()
log_file = list(uploaded.keys())[0]  # Get the filename of the uploaded file

# Read the content of the uploaded log file
with open(log_file, "r") as file:
    log_data = file.read()

# Define the AI model to use (Gemini 2.0 Pro)
model = genai.GenerativeModel("gemini-2.0-pro-exp-02-05")

# Ask the AI model to analyze the log file and identify potential threats
response = model.generate_content(f"Analyze this firewall log and identify potential threats:\n{log_data}")

# Print the AI-generated threat analysis report
print("\n--- Threat Analysis Report ---\n")
print(response.text)

# Write a prompt for extracting key security insights
threat_summary_prompt = """
Analyze the given firewall log and extract potential indicators of compromise (IOCs).
Look for:
- Suspicious IPs
- Unusual ports
- Frequent failed login attempts
- Signs of brute force attacks

Provide a structured summary in JSON format.
"""

# Ask the AI to generate a structured summary of potential threats
response = model.generate_content(f"{threat_summary_prompt}\n\n{log_data}")

# Print the extracted Indicators of Compromise (IOCs)
print("\n--- Indicators of Compromise (IOCs) ---\n")

# Dictionary to store failed login attempts by IP address
failed_attempts = {}

# Define a regex pattern to detect failed login attempts from log entries
pattern = r"Failed login from IP: (\d+\.\d+\.\d+\.\d+)"

# Search for failed login attempts in the log data
matches = re.findall(pattern, log_data)

# Count occurrences of each suspicious IP
for ip in matches:
    failed_attempts[ip] = failed_attempts.get(ip, 0) + 1

# Convert the failed attempts dictionary to a DataFrame for visualization
df = pd.DataFrame(list(failed_attempts.items()), columns=["IP", "Failed_Attempts"])

# Check if there are any failed attempts before plotting
if not df.empty:
    # Set up a bar chart to visualize suspicious login attempts
    plt.figure(figsize=(8,5))
    plt.bar(df["IP"], df["Failed_Attempts"], color="red")
    plt.xlabel("Suspicious IPs")  # Label for x-axis
    plt.ylabel("Failed Login Attempts")  # Label for y-axis
    plt.title("Detected Brute Force Attempts")  # Chart title
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()
else:
    print("No failed login attempts detected in the log file.")

# Save the AI-generated threat analysis to a text file
with open("threat_analysis.txt", "w") as file:
    file.write(response.text)

# download the generated threat analysis report in JSON format
files.download("threat_analysis.txt")


Saving log_file.txt to log_file.txt

--- Threat Analysis Report ---

Here's an analysis of the firewall log, identifying potential threats and explaining the reasoning:

**Key Observations and Threat Analysis:**

*   **Internal Network:** The log predominantly shows traffic originating from the `192.168.1.0/24` network, which is a standard private IP address range.  This suggests the log is from a firewall protecting an internal network.

*   **Blocked Connections - Potential Threats:** Several blocked connections stand out as potential security concerns:

    *   **`2023-03-15 06:27:58 BLOCK TCP 192.168.1.105 203.0.113.5 44347 22 48 S - SSH Attempt`:**  This is the *most significant* potential threat.  A machine inside your network (`192.168.1.105`) is attempting to establish an SSH connection (port 22) to an external IP address (`203.0.113.5`).  This could indicate:
        *   **Compromised Host:**  The internal machine (`.105`) might be infected with malware that is trying to conne

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>